In [8]:
from pathlib import Path
from monai.transforms import Compose, LoadImage, EnsureChannelFirst, Lambda, RepeatChannel
from monai.data import Dataset
import random


In [9]:
def data_load(data_dir, inspect=True, n_samples=3):
    """
    Create a MONAI Dataset, ensure channels come first, and optionally inspect sample properties.

    Parameters
    ----------
    data_dir : str or Path
        Root directory containing image files.
    inspect : bool
        Whether to print sample dataset information.
    n_samples : int
        Number of samples to inspect. samples are chosen randomly.

    Returns
    -------
    dataset : monai.data.Dataset
        Lazy-loading MONAI dataset.    
    """
    data_dir = Path(data_dir)
    image_paths = list(data_dir.rglob("*.jpg"))

    transforms = Compose([
        LoadImage(image_only=True),
        EnsureChannelFirst(),
    ])

    dataset = Dataset(
        data=image_paths,
        transform=transforms
    )

    # 🔍 Lightweight inspection
    if inspect and len(dataset) > 0:
        print(f"Dataset size: {len(dataset)} images")

        sample_indices = random.sample(
            range(len(dataset)),
            min(n_samples, len(dataset))
        )

        for idx in sample_indices:
            img = dataset[idx]
            print(
                f"Sample {idx}: "
                f"shape={tuple(img.shape)}, "
                f"dtype={img.dtype}, "
                f"min={float(img.min()):.2f}, "
                f"max={float(img.max()):.2f}"
            )

    return dataset

In [10]:
# test 
test_data=data_load(data_dir='../data/interim_deduped')

Dataset size: 14232 images
Sample 5739: shape=(3, 256, 256), dtype=torch.float32, min=0.00, max=255.00
Sample 8256: shape=(3, 256, 256), dtype=torch.float32, min=0.00, max=223.00
Sample 8463: shape=(3, 512, 512), dtype=torch.float32, min=0.00, max=188.00


In [36]:

def transform(dataset):
    """
    Return a new transformed Dataset
     a- convert to grayscale by averaging channels since some images have incononsistent channels
     b- repeat channels to have 3 channels again.
     c- resize to 224x224

    Parameters
    ----------
    dataset : monai.data.Dataset
        Existing dataset (paths will be reused).

    Returns
    -------
    dataset_transformed : monai.data.Dataset
        New dataset after applying the transformations.
    """
    image_paths = dataset.data  # reuse paths
    
    to_gray = Lambda(lambda x: x.mean(dim=0, keepdim=True))

    transforms = Compose([
        LoadImage(image_only=True),
        EnsureChannelFirst(),
        to_gray,
        RepeatChannel(3),
    ])

    dataset_transformed = Dataset(
        data=image_paths,
        transform=transforms
    )

    return dataset_transformed

In [39]:
test_data2 = transform(test_data)
test_data2[14000].shape

torch.Size([3, 1084, 717])